# GPT Practice

In [ ]:
# Check if torch can use cuda.
import torch

s1  = torch.cuda.is_available()
s2 = torch.cuda.device_count()
s3 = torch.cuda.current_device()
s4 = torch.cuda.get_device_name(0)
print(f'{s1}, {s2}, {s3}, {s4}')

In [ ]:
# Get dataset
import wget
from pathlib import Path

# Checks if dataset was already downloaded. Skips the download routine if dataset was detected.
dataset_path = Path.cwd() / 'input.txt'
if not dataset_path.exists():
    url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    filename = wget.download(url)
    print(f'Done downloading dataset {filename} from {url}')
else:
    print("Dataset already existed hence skipping download.")